In [22]:
pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=bf8bfa229cf81d2b90a23c267b2430f42b3cdf1218b2bacf597bec2925711c34
  Stored in directory: /Users/derek/Library/Caches/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
Note: you may need to restart the kernel to use updated packages.


In [183]:
import pandas as pd
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
import math
import requests
from sec_cik_mapper import StockMapper
from pathlib import Path


import os
import wget

In [13]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

In [9]:
df.head()

,adsh,tag,version,coreg,ddate,qtrs,uom,value,footnote
0,0001140361-22-040133,AcceleratedShareRepurchaseProgramAdjustment,us-gaap/2021,NaN,20220331,1,USD,-150000000.0,NaN
1,0001140361-22-040134,AcceleratedShareRepurchaseProgramAdjustment,us-gaap/2021,NaN,20220331,1,USD,-150000000.0,NaN
2,0001140361-22-040134,AcceleratedShareRepurchaseProgramAdjustment,us-gaap/2021,NaN,20220630,1,USD,0.0,NaN
3,0001628280-22-028255,AcceleratedShareRepurchaseProgramAdjustment,us-gaap/2021,NaN,20220930,1,USD,20010000.0,NaN
4,0000860413-22-000202,AcceleratedShareRepurchasesAdjustmentToRecorde...,us-gaap/2021,NaN,20220930,3,USD,197400000.0,NaN


In [28]:
cikdf = df1[df1.adsh.isin(df[df.tag.str.contains('^Net', na=False)]['adsh'])]
  

# fix period reporting date

In [32]:
cikdf['perint'] = cikdf['period'].fillna(19700101).astype('int')
cikdf['per'] = pd.to_datetime(cikdf['perint'], format='%Y%m%d')

<ipython-input-32-565ed8d5987d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cikdf['perint'] = cikdf['period'].fillna(19700101).astype('int')
<ipython-input-32-565ed8d5987d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cikdf['per'] = pd.to_datetime(cikdf['perint'], format='%Y%m%d')


In [33]:
cikdf.head()

,adsh,cik,name,sic,countryba,stprba,cityba,zipba,bas1,bas2,baph,countryma,stprma,cityma,zipma,mas1,mas2,countryinc,stprinc,ein,former,changed,afs,wksi,fye,form,period,fy,fp,filed,accepted,prevrpt,detail,instance,nciks,aciks,perint,per
0,0000014707-22-000072,14707,CALERES INC,3140.0,US,MO,ST LOUIS,63105,8300 MARYLAND AVE,P O BOX 29,3148544000,US,MO,ST LOUIS,63166,P O BOX 29,NaN,US,NY,430197190,BROWN SHOE CO INC,20030613.0,1-LAF,0,131.0,10-Q,20221031.0,2022.0,Q3,20221206,2022-12-06 14:50:00.0,0,1,cal-20221029x10q_htm.xml,1,NaN,20221031,2022-10-31
5,0000047217-22-000068,47217,HP INC,3570.0,US,CA,PALO ALTO,94304,1501 PAGE MILL ROAD,NaN,6508571501,US,CA,PALO ALTO,94304,1501 PAGE MILL ROAD,NaN,US,DE,941081436,HEWLETT PACKARD CO,19920703.0,1-LAF,0,1031.0,10-K,20221031.0,2022.0,FY,20221206,2022-12-06 17:04:00.0,0,1,hpq-20221031_htm.xml,1,NaN,20221031,2022-10-31
6,0000048465-22-000051,48465,HORMEL FOODS CORP /DE/,2011.0,US,MN,AUSTIN,55912-3680,1 HORMEL PL,NaN,(507) 437-5611,US,MN,AUSTIN,55912-3680,1 HORMEL PLACE,NaN,US,DE,410319970,HORMEL GEO A & CO,19920703.0,1-LAF,1,1031.0,10-K,20221031.0,2022.0,FY,20221206,2022-12-06 13:35:00.0,0,1,hrl-20221030_htm.xml,1,NaN,20221031,2022-10-31
8,0000074046-22-000096,74046,OIL-DRI CORP OF AMERICA,3990.0,US,IL,CHICAGO,60611,410 NORTH MICHIGAN AVENUE,SUITE 400,3123211515,US,IL,CHICAGO,60611,410 NORTH MICHIGAN AVENUE,SUITE 400,US,DE,362048898,OIL-DRI CORP OF AMERICA,20170417.0,2-ACC,0,731.0,10-Q,20221031.0,2023.0,Q1,20221206,2022-12-06 16:14:00.0,0,1,odc-20221031_htm.xml,1,NaN,20221031,2022-10-31
9,0000080420-22-000036,80420,POWELL INDUSTRIES INC,3613.0,US,TX,HOUSTON,77075,8550 MOSLEY DR,POST OFFICE BOX 12818,7139446900,US,TX,HOUSTON,77075,8550 MOSLEY DRIVE P O BOX 12818,8550 MOSLEY DRIVE P O BOX 12818,US,NV,880106100,PROCESS SYSTEMS INC,19780926.0,2-ACC,0,930.0,10-K,20220930.0,2022.0,FY,20221206,2022-12-06 15:42:00.0,0,1,powl-20220930_htm.xml,1,NaN,20220930,2022-09-30


In [115]:

def importit(p, path):
    from zipfile import ZipFile
    ZipFile(path).extractall(p)
    print('unziping '+p)

    df = pd.read_table('num.txt')
    df1 = pd.read_table('sub.txt')
    
    return df,df1
    
def load_qtr_data(yrs, qtrs=[1,5]):
    print(yrs)
    fdf1 = pd.DataFrame()
    fdf = pd.DataFrame()
    for yr in yrs:
        print(yr)
        
        for qtr in qtrs:
            print('Downloading Q'+str(qtr)+'-'+str(yr))
            fnme = str(yr)+'q'+str(qtr)+'.zip'
            url = 'https://www.sec.gov/files/dera/data/financial-statement-data-sets/'+fnme

            path = os.getcwd() + '/'+fnme
            p = os.getcwd()

            wget.download(url)
            print('downloaded!')
            df, df1 = importit(p, path) 
            
            if len(df) > 0:
                fdf = pd.concat([fdf,df])
            
            if len(df1) >0:
                fdf1 = pd.concat([fdf1,df1])
            print('done downloading')
            return fdf, fdf1
            
df, df1 = load_qtr_data(range(2022,2023),range(3,5)) #years will exclude first year #1,5 = qtrs 1-4

range(2022, 2023)
2022
downloaded!
unziping /Users/derek/Documents/Misc
done downloading


In [116]:
#search rows for tags beginning with Net
#cikdf = df1[df1.adsh.isin(df[df.tag.str.contains('NetIncomeLoss', na=False)]['adsh'])]
#Get NIL only!
cikdf = df1[df1.adsh.isin(df[df.tag == 'NetIncomeLoss']['adsh'])]

#get the ciks that have net income
cikdf['perint'] = cikdf['period'].fillna(19700101).astype('int')
cikdf['per'] = pd.to_datetime(cikdf['perint'], format='%Y%m%d')

#filter 
filtered = df[df.tag.str.contains('^NetI', na=False)]
filtered.head()

<ipython-input-116-cc471238e1e9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cikdf['perint'] = cikdf['period'].fillna(19700101).astype('int')
<ipython-input-116-cc471238e1e9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cikdf['per'] = pd.to_datetime(cikdf['perint'], format='%Y%m%d')


,adsh,tag,version,coreg,ddate,qtrs,uom,value,footnote
274257,0000072971-22-000174,NetIncomeLoss,us-gaap/2021,NaN,20220630,2,USD,6.790000e+09,NaN
274258,0000072971-22-000174,NetIncomeLoss,us-gaap/2021,NaN,20210630,2,USD,1.067600e+10,NaN
274259,0000072971-22-000174,NetIncomeLoss,us-gaap/2021,NaN,20220630,1,USD,3.119000e+09,NaN
274260,0000072971-22-000174,NetIncomeLoss,us-gaap/2021,NaN,20210630,1,USD,6.040000e+09,NaN
274261,0000072909-22-000010,NetIncomeLoss,us-gaap/2021,NaN,20210630,1,USD,1.400000e+07,NaN


In [117]:
ni = filtered.merge(cikdf, how='left')
ni.head()

,adsh,tag,version,coreg,ddate,qtrs,uom,value,footnote,cik,name,sic,countryba,stprba,cityba,zipba,bas1,bas2,baph,countryma,stprma,cityma,zipma,mas1,mas2,countryinc,stprinc,ein,former,changed,afs,wksi,fye,form,period,fy,fp,filed,accepted,prevrpt,detail,instance,nciks,aciks,perint,per
0,0000072971-22-000174,NetIncomeLoss,us-gaap/2021,NaN,20220630,2,USD,6.790000e+09,NaN,72971.0,WELLS FARGO & COMPANY/MN,6021.0,US,CA,SAN FRANCISCO,94163,420 MONTGOMERY STREET,NaN,6126671234,US,CA,SAN FRANCISCO,94163,420 MONTGOMERY STREET,NaN,US,DE,410449260.0,WELLS FARGO & CO/MN,19981103.0,1-LAF,0.0,1231.0,10-Q,20220630.0,2022.0,Q2,20220801.0,2022-08-01 17:03:00.0,0.0,1.0,wfc-20220630_htm.xml,1.0,NaN,20220630.0,2022-06-30
1,0000072971-22-000174,NetIncomeLoss,us-gaap/2021,NaN,20210630,2,USD,1.067600e+10,NaN,72971.0,WELLS FARGO & COMPANY/MN,6021.0,US,CA,SAN FRANCISCO,94163,420 MONTGOMERY STREET,NaN,6126671234,US,CA,SAN FRANCISCO,94163,420 MONTGOMERY STREET,NaN,US,DE,410449260.0,WELLS FARGO & CO/MN,19981103.0,1-LAF,0.0,1231.0,10-Q,20220630.0,2022.0,Q2,20220801.0,2022-08-01 17:03:00.0,0.0,1.0,wfc-20220630_htm.xml,1.0,NaN,20220630.0,2022-06-30
2,0000072971-22-000174,NetIncomeLoss,us-gaap/2021,NaN,20220630,1,USD,3.119000e+09,NaN,72971.0,WELLS FARGO & COMPANY/MN,6021.0,US,CA,SAN FRANCISCO,94163,420 MONTGOMERY STREET,NaN,6126671234,US,CA,SAN FRANCISCO,94163,420 MONTGOMERY STREET,NaN,US,DE,410449260.0,WELLS FARGO & CO/MN,19981103.0,1-LAF,0.0,1231.0,10-Q,20220630.0,2022.0,Q2,20220801.0,2022-08-01 17:03:00.0,0.0,1.0,wfc-20220630_htm.xml,1.0,NaN,20220630.0,2022-06-30
3,0000072971-22-000174,NetIncomeLoss,us-gaap/2021,NaN,20210630,1,USD,6.040000e+09,NaN,72971.0,WELLS FARGO & COMPANY/MN,6021.0,US,CA,SAN FRANCISCO,94163,420 MONTGOMERY STREET,NaN,6126671234,US,CA,SAN FRANCISCO,94163,420 MONTGOMERY STREET,NaN,US,DE,410449260.0,WELLS FARGO & CO/MN,19981103.0,1-LAF,0.0,1231.0,10-Q,20220630.0,2022.0,Q2,20220801.0,2022-08-01 17:03:00.0,0.0,1.0,wfc-20220630_htm.xml,1.0,NaN,20220630.0,2022-06-30
4,0000072909-22-000010,NetIncomeLoss,us-gaap/2021,NaN,20210630,1,USD,1.400000e+07,NaN,72909.0,NORTHERN STATES POWER CO /WI/,4931.0,US,WI,EAU CLAIRE,54702,1414 W HAMILTON AVE,NaN,7158392621,US,WI,EAU CLAIRE,54702-008,P O BOX 8,NaN,US,WI,390508315.0,NaN,NaN,4-NON,0.0,1231.0,10-Q,20220630.0,2022.0,Q2,20220728.0,2022-07-28 12:21:00.0,0.0,1.0,nspw-20220630_htm.xml,1.0,NaN,20220630.0,2022-06-30


In [118]:
len(ni)

57505

In [130]:
uom = 'USD' #monetary units!
coreg = None #means it is the parent company!
version = 'usGaap version'
qtrs = 'which quarter?' #0 mean spoint in time
iord = ''#If abstract=1, then NULL; otherwise, “I” if the value is a point-in time, or “D” if the value is a duration.
#I only want a single qtr at a time
ni[['adsh','cik','coreg','name','version','tag','per','value','former','ddate','accepted','form','qtrs','uom']].head()


,adsh,cik,coreg,name,version,tag,per,value,former,ddate,accepted,form,qtrs,uom
0,0000072971-22-000174,72971.0,NaN,WELLS FARGO & COMPANY/MN,us-gaap/2021,NetIncomeLoss,2022-06-30,6.790000e+09,WELLS FARGO & CO/MN,20220630,2022-08-01 17:03:00.0,10-Q,2,USD
1,0000072971-22-000174,72971.0,NaN,WELLS FARGO & COMPANY/MN,us-gaap/2021,NetIncomeLoss,2022-06-30,1.067600e+10,WELLS FARGO & CO/MN,20210630,2022-08-01 17:03:00.0,10-Q,2,USD
2,0000072971-22-000174,72971.0,NaN,WELLS FARGO & COMPANY/MN,us-gaap/2021,NetIncomeLoss,2022-06-30,3.119000e+09,WELLS FARGO & CO/MN,20220630,2022-08-01 17:03:00.0,10-Q,1,USD
3,0000072971-22-000174,72971.0,NaN,WELLS FARGO & COMPANY/MN,us-gaap/2021,NetIncomeLoss,2022-06-30,6.040000e+09,WELLS FARGO & CO/MN,20210630,2022-08-01 17:03:00.0,10-Q,1,USD
4,0000072909-22-000010,72909.0,NaN,NORTHERN STATES POWER CO /WI/,us-gaap/2021,NetIncomeLoss,2022-06-30,1.400000e+07,NaN,20210630,2022-07-28 12:21:00.0,10-Q,1,USD


In [121]:
cikdf.columns

Index(['adsh', 'cik', 'name', 'sic', 'countryba', 'stprba', 'cityba', 'zipba',
       'bas1', 'bas2', 'baph', 'countryma', 'stprma', 'cityma', 'zipma',
       'mas1', 'mas2', 'countryinc', 'stprinc', 'ein', 'former', 'changed',
       'afs', 'wksi', 'fye', 'form', 'period', 'fy', 'fp', 'filed', 'accepted',
       'prevrpt', 'detail', 'instance', 'nciks', 'aciks', 'perint', 'per'],
      dtype='object')

In [146]:
ni.columns

Index(['adsh', 'tag', 'version', 'coreg', 'ddate', 'qtrs', 'uom', 'value',
       'footnote', 'cik', 'name', 'sic', 'countryba', 'stprba', 'cityba',
       'zipba', 'bas1', 'bas2', 'baph', 'countryma', 'stprma', 'cityma',
       'zipma', 'mas1', 'mas2', 'countryinc', 'stprinc', 'ein', 'former',
       'changed', 'afs', 'wksi', 'fye', 'form', 'period', 'fy', 'fp', 'filed',
       'accepted', 'prevrpt', 'detail', 'instance', 'nciks', 'aciks', 'perint',
       'per'],
      dtype='object')

In [125]:

mapper = StockMapper()
dicti = mapper.ticker_to_cik
dicti['AAPL']

'0000320193'

In [ ]:
## As we can see below the mapper.tik_to_ticker doesn't have many of the cik tickers available in the downloads
# from the SEC.

#...so how can we get the tickers? for example: 'NORTHERN STATES POWER CO /WI/'

In [191]:

def iterate_it(cikdf, limit):
    symbols_avail = []
    cik12 = mapper.cik_to_tickers
    errors = []
    errors_c = {}
    
    j = 0
    for r in cikdf.iterrows():
        
        #print(r[1]['cik'])
        try:
            ciknm = r[1]['cik']
        except:
            msg = 'no cik in this row!/n'
            msg += r
            errors.append(msg)
        
        if ciknm != 0:
            #print('passed test')
            #print(type(ciknm))
            if not math.isnan(ciknm):
                cik_num_translated = str(int(ciknm)).rjust(10, '0')
            else:
                break
            #print(cik_num_translated)
            if cik_num_translated in cik12:
                sym = cik12[cik_num_translated]
                #print(sym)
                for s in sym:
                    if s not in symbols_avail:
                        symbols_avail.append(s)
            else:
                if ciknm not in errors_c:
                    arr = {'cik': r[1]['name'],'err':str(ciknm) + ' has no symbol in the lib.'}
            
                    errors_c[int(ciknm)] = arr
                   
        if j >= limit:
            break
        ciknm = 0
        j+=1
    return symbols_avail, errors_c, errors
    
    
ss, err_c, err = iterate_it(ni, len(ni))
print(err_c)
print(ss)

{72909: {'cik': 'NORTHERN STATES POWER CO /WI/', 'err': '72909.0 has no symbol in the lib.'}, 54480: {'cik': 'KANSAS CITY SOUTHERN', 'err': '54480.0 has no symbol in the lib.'}, 38009: {'cik': 'FORD MOTOR CREDIT CO LLC', 'err': '38009.0 has no symbol in the lib.'}, 722723: {'cik': 'HANGER, INC.', 'err': '722723.0 has no symbol in the lib.'}, 92521: {'cik': 'SOUTHWESTERN PUBLIC SERVICE CO', 'err': '92521.0 has no symbol in the lib.'}, 81018: {'cik': 'PUBLIC SERVICE CO OF COLORADO', 'err': '81018.0 has no symbol in the lib.'}, 107833: {'cik': 'WISCONSIN PUBLIC SERVICE CORP', 'err': '107833.0 has no symbol in the lib.'}, 99250: {'cik': 'TRANSCONTINENTAL GAS PIPE LINE COMPANY, LLC', 'err': '99250.0 has no symbol in the lib.'}, 110019: {'cik': 'NORTHWEST PIPELINE LLC', 'err': '110019.0 has no symbol in the lib.'}, 205402: {'cik': 'GRAYBAR ELECTRIC CO INC', 'err': '205402.0 has no symbol in the lib.'}, 728391: {'cik': 'IPALCO ENTERPRISES, INC.', 'err': '728391.0 has no symbol in the lib.'}, 

In [196]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Let's try searching edgar itself to see what can be returned....not ideal

In [206]:
from bs4 import BeautifulSoup

In [208]:
tst = requests.get('https://www.edgarcompany.sec.gov/servlet/CompanyDBSearch?start_row=-1&end_row=-1&main_back=1&cik=0000072909&company_name=&reporting_file_number=&series_id=&series_name=&class_contract_id=&class_contract_name=&state_country=NONE&city=&state_incorporation=NONE&zip_code=&last_update_from=&last_update_to=&page=summary&submit_button=View+Summary')
soup = BeautifulSoup(tst.content, 'html.parser')
table = soup.find_all('table')[4]


<table border="0" width="753">
<tr>
<td>
<table border="0">
<tr>
<td style="color:blue; font-size:12pt" valign="TOP">1 company was found.</td>
</tr>
</table>
</td>
</tr>
<tr><td style="font-size:2pt"> </td></tr>
<tr>
<td>
<table border="0" width="100%">
<tr>
<td width="25"> </td>
<td colspan="4">
<hr size="3"/>
</td>
</tr>
<tr>
<td width="25"> </td>
<td class="c1" valign="TOP" width="25">1.</td>
<td class="c1" colspan="4"><a href="/servlet/CompanyDBSearch?page=detailed&amp;cik=0000072909&amp;main_back=2">NORTHERN STATES POWER CO /WI/</a> </td>
</tr>
<tr>
<td class="c1" width="25"> </td>
<td class="c1" width="25"> </td>
<td class="c1" width="225">CIK:  0000072909</td>
<td class="c1" width="425">State of Incorporation:  WISCONSIN</td>
<td class="c1" width="25"> </td>
<td class="c1" width="25"> </td>
</tr>
<tr>
<td class="c1" width="25"> </td>
<td class="c1" width="25"> </td>
<td class="c1" width="225">Date of Last Update:  10/25/19</td>
<td class="c1" width="425">Regulated Entity Type:  

In [213]:
cik_name = table.find_all('td')
print(cik_name)


[<td>
<table border="0">
<tr>
<td style="color:blue; font-size:12pt" valign="TOP">3 companies were found.</td>
</tr>
</table>
</td>, <td style="color:blue; font-size:12pt" valign="TOP">3 companies were found.</td>, <td style="font-size:2pt"> </td>, <td>
<table border="0" width="100%">
<tr>
<td width="25"> </td>
<td colspan="4">
<hr size="3"/>
</td>
</tr>
<tr>
<td width="25"> </td>
<td class="c1" valign="TOP" width="25">1.</td>
<td class="c1" colspan="4"><a href="/servlet/CompanyDBSearch?page=detailed&amp;cik=0001441084&amp;main_back=2">Redbox Automated Retail, LLC</a> </td>
</tr>
<tr>
<td class="c1" width="25"> </td>
<td class="c1" width="25"> </td>
<td class="c1" width="225">CIK:  0001441084</td>
<td class="c1" width="425">State of Incorporation:  DELAWARE</td>
<td class="c1" width="25"> </td>
<td class="c1" width="25"> </td>
</tr>
<tr>
<td class="c1" width="25"> </td>
<td class="c1" width="25"> </td>
<td class="c1" width="225">Date of Last Update:  09/06/13</td>
<td class="c1" width=

In [211]:
tst = requests.get('https://www.edgarcompany.sec.gov/servlet/CompanyDBSearch?start_row=-1&end_row=-1&main_back=1&cik=&company_name=Redbox&reporting_file_number=&series_id=&series_name=&class_contract_id=&class_contract_name=&state_country=NONE&city=&state_incorporation=NONE&zip_code=&last_update_from=&last_update_to=&page=summary&submit_button=View+Summary')
soup = BeautifulSoup(tst.content, 'html.parser')
table = soup.find_all('table')[4]

In [210]:
import re, requests
headers = {"user-agent": "Safari"}

def getCIKs(TICKERS):
    URL = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany'
    CIK_RE = re.compile(r'.*CIK=(\d{10}).*')    
    cik_dict = {}
    for ticker in TICKERS:
        f = requests.get(URL.format(ticker),headers= headers, stream = True)
        results = CIK_RE.findall(f.text)
        if len(results):
            results[0] = int(re.sub('\.[0]*', '.', results[0]))
            cik_dict[str(ticker).upper()] = str(results[0])
    f = open('cik_dict', 'w')
    f.close()
    return(cik_dict)

getCIKs(['wmt','amzn','nflx'])
# returns:
# {'WMT': '104169', 'AMZN': '1018724', 'NFLX': '1065280'}

{}

In [193]:
pd.DataFrame.from_dict(err_c).T

,cik,err
72909,NORTHERN STATES POWER CO /WI/,72909.0 has no symbol in the lib.
54480,KANSAS CITY SOUTHERN,54480.0 has no symbol in the lib.
38009,FORD MOTOR CREDIT CO LLC,38009.0 has no symbol in the lib.
722723,"HANGER, INC.",722723.0 has no symbol in the lib.
92521,SOUTHWESTERN PUBLIC SERVICE CO,92521.0 has no symbol in the lib.
81018,PUBLIC SERVICE CO OF COLORADO,81018.0 has no symbol in the lib.
107833,WISCONSIN PUBLIC SERVICE CORP,107833.0 has no symbol in the lib.
99250,"TRANSCONTINENTAL GAS PIPE LINE COMPANY, LLC",99250.0 has no symbol in the lib.
110019,NORTHWEST PIPELINE LLC,110019.0 has no symbol in the lib.
205402,GRAYBAR ELECTRIC CO INC,205402.0 has no symbol in the lib.


In [ ]:
## found this: Public FTP! for NASDAQ
#ftp://ftp.nasdaqtrader.com/symboldirectory...now to find out how to DL the files...

In [215]:
from ftplib import FTP
ftp = FTP('ftp://ftp.nasdaqtrader.com')  # connect to host, default port
ftp.login() 

gaierror: [Errno 8] nodename nor servname provided, or not known

In [ ]:
#!/usr/bin/env python
 
import ftplib
import os
import re
 
# Connect to ftp.nasdaqtrader.com
ftp = ftplib.FTP('ftp.nasdaqtrader.com', 'anonymous', 'anonymous@debian.org')
 
# Download files nasdaqlisted.txt and otherlisted.txt from ftp.nasdaqtrader.com
for ficheiro in ["nasdaqlisted.txt", "otherlisted.txt"]:
        ftp.cwd("/SymbolDirectory")
        localfile = open(ficheiro, 'wb')
        ftp.retrbinary('RETR ' + ficheiro, localfile.write)
        localfile.close()
ftp.quit()
 
# Grep for common stock in nasdaqlisted.txt and otherlisted.txt
for ficheiro in ["nasdaqlisted.txt", "otherlisted.txt"]:
        localfile = open(ficheiro, 'r')
        for line in localfile:
                if re.search("Common Stock", line):
                        ticker = line.split("|")[0]
                        # Append tickers to file tickers.txt
                        open("tickers.txt","a+").write(ticker + "\n")


In [ ]:
## Try Yahoo finance

In [216]:
import yfinance as yf
# Request historical data for past 5 years
data = yf.Ticker("NVDA").history(period='5y')
# Show info
print(data.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1258 entries, 2018-05-17 00:00:00-04:00 to 2023-05-16 00:00:00-04:00
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          1258 non-null   float64
 1   High          1258 non-null   float64
 2   Low           1258 non-null   float64
 3   Close         1258 non-null   float64
 4   Volume        1258 non-null   int64  
 5   Dividends     1258 non-null   float64
 6   Stock Splits  1258 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 78.6 KB
None
